# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_memoris import lexicon_memoris, lithology_memoris
from utils.io import dataframe_viewer
from utils.orm import boreholes_from_dataframe
from utils.visual import legend_from_attributes
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/final_data/'

In [3]:
df = pd.read_csv(save_dir+'Final_dataset.csv', sep=',')

In [4]:
dataframe_viewer(df, rows=5, un_val='ID')

Rows : 112, columns : 131, Unique values on col 'ID': 46


interactive(children=(IntSlider(value=5, description='rows', max=112, min=5, readout=False), IntSlider(value=1…

## Data Checking

In [5]:
df.rename({'Ech_base':'Base_intv', 'Ech_top':'Top_intv', 'Ep_litho':'Epais_intv', 'Description':'Lithology'}
          , axis='columns', inplace=True)

In [6]:
df['Epais_intv'] = df['Base_intv'] - df['Top_intv']

In [7]:
df['Diam_for'] = df['Diam_for']/1000
df.insert(list(df.columns).index('Long_for')+1, 'Diam_for', df.pop('Diam_for'))

In [8]:
dataframe_viewer(df, rows=3, un_val='ID_ech')

Rows : 112, columns : 132, Unique values on col 'ID_ech': 112


interactive(children=(IntSlider(value=3, description='rows', max=112, min=3, readout=False), IntSlider(value=1…

## Create Boreholes from dataframes or files

In [9]:
id_list = ['302','205','207','214']
#df = df.query('ID in @id_list')
#df.reset_index(drop=True, inplace=True)
dataframe_viewer(df.query('ID in @id_list'))

Rows : 15, columns : 132


interactive(children=(IntSlider(value=10, description='rows', max=15, min=10, readout=False), IntSlider(value=…

In [10]:
boreholes, components, link_intv_comp = boreholes_from_dataframe(df, verbose=False,
                                                attributes=['Lithology'], 
                                                symbols={'lithology':{'lexicon':lexicon_memoris}}, 
                                                diameter='Diam_for', thickness='Epais_intv')


Dataframe processing...

|__ID:'201'
Summary : [Striplog(2 Intervals, start=0.0, stop=0.8999999999999999)]

|__ID:'205'
Summary : [Striplog(4 Intervals, start=0.0, stop=1.5)]

|__ID:'208'
Summary : [Striplog(4 Intervals, start=0.0, stop=1.4999999999999996)]

|__ID:'207'
Summary : [Striplog(5 Intervals, start=0.0, stop=1.8000000000000007)]

|__ID:'212'
Summary : [Striplog(4 Intervals, start=0.0, stop=1.4999999999999991)]

|__ID:'214'
Summary : [Striplog(4 Intervals, start=0.0, stop=1.6999999999999997)]

|__ID:'217'
Summary : [Striplog(4 Intervals, start=0.0, stop=1.5000000000000004)]

|__ID:'221'
Error : No value matching with 'Vide ' in given lexicon
Summary : [Striplog(1 Intervals, start=0.0, stop=0.4)]

|__ID:'223'
Summary : [Striplog(2 Intervals, start=0.0, stop=0.8)]

|__ID:'?2'
Summary : [Striplog(1 Intervals, start=0.0, stop=0.3999999999999999)]

|__ID:'?1'
Summary : [Striplog(1 Intervals, start=0.0, stop=0.3999999999999999)]

|__ID:'224'
Summary : [Striplog(1 Intervals, start=0

/home/yanathan/Projects/GSDMA/utils/orm.py:126: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="components"' to the 'ComponentOrm.intervals' relationship.
  boreholes_orm.append(BoreholeOrm(id=bh_name, date=bh_date))
/home/yanathan/Projects/GSDMA/utils/orm.py:126: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.intv_id, which conflic

In [11]:
len(boreholes), len(components)

(46, 8)

## Create a project and save data in a database

In [12]:
remove('tmp_files/tfe_orm_db.db')

In [13]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [14]:
Base.metadata.create_all(engine)

2021-09-22 03:16:06,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 03:16:06,352 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-09-22 03:16:06,353 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-22 03:16:06,354 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-09-22 03:16:06,354 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-22 03:16:06,355 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-09-22 03:16:06,356 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-22 03:16:06,356 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-09-22 03:16:06,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-22 03:16:06,357 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Components")
2021-09-22 03:16:06,358 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-22 03:16:06,358 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Components")
2021-09-22 03:16:06,359 INFO sqlalchemy.engine.Engin

In [15]:
Session = sessionmaker(bind=engine)
session = Session()

## Create a legend dictionary for the project

In [16]:
legend_dict = legend_from_attributes([('lithology', lithology_memoris)])

In [17]:
p = Project(session, legend_dict=legend_dict, lexicon=lexicon_memoris)

2021-09-22 03:16:06,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 03:16:06,422 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-09-22 03:16:06,424 INFO sqlalchemy.engine.Engine [generated in 0.00279s] ()


#### Add components and boreholes into the project

In [18]:
p.add_components(components)

2021-09-22 03:16:06,432 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-09-22 03:16:06,434 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ((0, 'Remblais'), (1, 'Boue'), (2, '_'), (3, 'Schistes'), (4, 'Limon'), (5, 'Limons'), (6, 'Béton'), (7, 'Alluvions'))
2021-09-22 03:16:06,435 INFO sqlalchemy.engine.Engine COMMIT
2021-09-22 03:16:06,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 03:16:06,439 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-09-22 03:16:06,440 INFO sqlalchemy.engine.Engine [cached since 0.01827s ago] ()


In [19]:
for bh in boreholes:
    p.add_borehole(bh)

2021-09-22 03:16:06,445 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-09-22 03:16:06,447 INFO sqlalchemy.engine.Engine [generated in 0.00212s] ('201', None, 0.8999999999999999, 0.075)
2021-09-22 03:16:06,450 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-09-22 03:16:06,451 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ((0, 101.926886, 101.926886, 101.926886, 152890.245758, 122596.47426), (1, 101.426886, 101.426886, 101.426886, 152890.245758, 122596.47426), (2, 101.426886, 101.426886, 101.426886, 152890.245758, 122596.47426), (3, 101.02688599999999, 101.02688599999999, 101.02688599999999, 152890.245758, 122596.47426))
2021-09-22 03:16:06,452 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?)
2021-09-22 03:16:06,453 INFO sqlalchemy.engine.Engine [g

2021-09-22 03:16:06,509 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-09-22 03:16:06,510 INFO sqlalchemy.engine.Engine [cached since 0.04247s ago] (8,)
2021-09-22 03:16:06,511 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-09-22 03:16:06,512 INFO sqlalchemy.engine.Engine [cached since 0.04178s ago] (9,)
2021-09-22 03:16:06,514 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "

2021-09-22 03:16:06,569 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-09-22 03:16:06,570 INFO sqlalchemy.engine.Engine [cached since 0.1198s ago] ((20, 101.889132, 101.889132, 101.889132, 152892.92499700002, 122592.662277), (21, 101.489132, 101.489132, 101.489132, 152892.92499700002, 122592.662277), (22, 101.489132, 101.489132, 101.489132, 152892.92499700002, 122592.662277), (23, 101.089132, 101.089132, 101.089132, 152892.92499700002, 122592.662277), (24, 101.089132, 101.089132, 101.089132, 152892.92499700002, 122592.662277), (25, 100.689132, 100.689132, 100.689132, 152892.92499700002, 122592.662277), (26, 100.689132, 100.689132, 100.689132, 152892.92499700002, 122592.662277), (27, 100.489132, 100.489132, 100.489132, 152892.92499700002, 122592.662277), (28, 100.489132, 100.489132, 100.489132, 152892.92499700002, 122592.662277), (29, 100.089132, 100.089132, 100.089132, 152892.92499700002, 122592.662277))
2021-09-22 

2021-09-22 03:16:06,637 INFO sqlalchemy.engine.Engine COMMIT
2021-09-22 03:16:06,641 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 03:16:06,642 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-09-22 03:16:06,643 INFO sqlalchemy.engine.Engine [cached since 0.2213s ago] ()
2021-09-22 03:16:06,644 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id", "Intervals".data_id AS "Intervals_data_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-09-22 03:16:06,645 INFO sqlalchemy.engine.Engine [cached since 0.1814s ago] ('212',)
2021-09-22 03:16:06,647 

2021-09-22 03:16:06,698 INFO sqlalchemy.engine.Engine [cached since 0.2309s ago] (40,)
2021-09-22 03:16:06,700 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-09-22 03:16:06,701 INFO sqlalchemy.engine.Engine [cached since 0.2302s ago] (41,)
2021-09-22 03:16:06,703 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-09-22 03:16:06,704 INFO sqlalchemy.engine.Engine [cached since 0.2366s ago] (42,)
2021-09-22 03:16:06,706 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_i

Borehole geometry created successfully !
2021-09-22 03:16:06,775 INFO sqlalchemy.engine.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-09-22 03:16:06,776 INFO sqlalchemy.engine.Engine [cached since 0.2932s ago] (101.315009, '217')
2021-09-22 03:16:06,776 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-09-22 03:16:06,776 INFO sqlalchemy.engine.Engine [cached since 0.3313s ago] ('221', None, 0.4, 0.075)
2021-09-22 03:16:06,777 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-09-22 03:16:06,777 INFO sqlalchemy.engine.Engine [cached since 0.3273s ago] ((54, 101.798666, 101.798666, 101.798666, 152882.412821, 122584.570092), (55, 101.39866599999999, 101.39866599999999, 101.39866599999999, 152882.412821, 122584.570092))
2021-09-22 03:16:06,778 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top

2021-09-22 03:16:06,850 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 03:16:06,851 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-09-22 03:16:06,851 INFO sqlalchemy.engine.Engine [cached since 0.4299s ago] ()
2021-09-22 03:16:06,854 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id", "Intervals".data_id AS "Intervals_data_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-09-22 03:16:06,855 INFO sqlalchemy.engine.Engine [cached since 0.391s ago] ('?2',)
2021-09-22 03:16:06,857 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positio

2021-09-22 03:16:06,931 INFO sqlalchemy.engine.Engine [cached since 0.4808s ago] ((66, 101.669021, 101.669021, 101.669021, 152881.112067, 122580.961648), (67, 101.269021, 101.269021, 101.269021, 152881.112067, 122580.961648), (68, 101.269021, 101.269021, 101.269021, 152881.112067, 122580.961648), (69, 100.769021, 100.769021, 100.769021, 152881.112067, 122580.961648), (70, 100.769021, 100.769021, 100.769021, 152881.112067, 122580.961648), (71, 100.369021, 100.369021, 100.369021, 152881.112067, 122580.961648), (72, 100.369021, 100.369021, 100.369021, 152881.112067, 122580.961648), (73, 99.969021, 99.969021, 99.969021, 152881.112067, 122580.961648)  ... displaying 10 of 12 total bound parameter sets ...  (76, 99.469021, 99.469021, 99.469021, 152881.112067, 122580.961648), (77, 98.969021, 98.969021, 98.969021, 152881.112067, 122580.961648))
2021-09-22 03:16:06,932 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id, data_id) VA

2021-09-22 03:16:06,985 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?)
2021-09-22 03:16:06,986 INFO sqlalchemy.engine.Engine [cached since 0.5335s ago] ((39, '226', 0, '{"lithology": "remblais"}', 78, 79, None), (40, '226', 1, '{"lithology": "remblais"}', 80, 81, None), (41, '226', 2, '{"lithology": "remblais"}', 82, 83, None), (42, '226', 3, '{"lithology": "remblais"}', 84, 85, None))
2021-09-22 03:16:06,987 INFO sqlalchemy.engine.Engine COMMIT
2021-09-22 03:16:06,992 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-22 03:16:06,993 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-09-22 03:16:06,994 INFO sqlalchemy.engine.Engine [cached since 0.572s ago] ()
2021-09-22 03:16:06,996 INFO sqlalchemy.engine.Engi

PositionError: You must provide middle, or upper and lower.

In [ ]:
p.add_link_components_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes_orm))

In [ ]:
p.boreholes_orm

In [ ]:
p.refresh()

In [ ]:
session.close()

### update project legend dict

In [ ]:
p.update_legend_cmap(update_project_legend=True, update_all_attrib=True, verbose=False)

In [ ]:
p.legend_dict['lithology']['legend'] # looks like cmap and values combination

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
n = 2
df.query(f'ID=="{p.boreholes_3d[n].name}"')[['ID', 'Top_intv', 'Base_intv', 'Lithology']]

In [ ]:
p.boreholes_3d[n].intervals[0].components #[0].lithology

In [ ]:
p.boreholes_3d[n].log_plot() #verbose=True)

### 2D/3D display of project boreholes

In [ ]:
p.plot2d()

In [ ]:
p.legend_dict['lithology']['legend']

In [ ]:
p.plot3d(repr_attribute='lithology', labels_size=15, bg_color=["royalblue", "aliceblue"], 
         window_size=(800, 600), verbose=True)